In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep as mp
import gmeep as gm

# MPB modes

MPB has periodic boundary conditions.


In [ ]:
# plot TE mode (plot_mode_order=1)
r = gm.find_modes(wg_width=0.4, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=1, parity=mp.NO_PARITY)
r

In [ ]:
gm.plot_modes(r['mode_solver'])

In [ ]:
# plot TM mode (plot_mode_order=2)
r = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=2, parity=mp.NO_PARITY)
r

In [ ]:
gm.plot_modes(r['mode_solver'])

In [ ]:
# plot TM mode (plot_mode_order=1)
import gmeep as gm
import meep as mp

r = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, parity=mp.EVEN_Y, mode_number=1)
r

In [ ]:
gm.plot_modes(r['mode_solver'])

In [ ]:
# TM mode has some issue if no parity is applied
#neff, ng = gm.find_modes(wg_width=0.5, ncore=3.47, nclad=1.44, wg_thickness=.22, plot=True, res=32, sx=6, sy=6, mode_number=2)
#neff, ng

In [ ]:
import gmeep as gm
import meep as mp
r = gm.find_modes(wg_width=0.8, ncore=3.47, nclad=1.44, wg_thickness=.22, res=32, sx=6, sy=6, mode_number=1, parity=mp.EVEN_Y)
r

In [ ]:
wg_widths = np.arange(200, 2000, 100)*1e-3
n_te0 = [gm.find_modes(wg_width=wg_width, ncore=3.55, wg_thickness=.22) for wg_width in wg_widths]

In [ ]:
neffs_te0 = [ni['neff'] for ni in n_te0]
ngs_te0 = [ni['ng'] for ni in n_te0]

plt.plot(wg_widths, neffs_te0, '.-')
plt.xlabel('wg_width')
plt.ylabel('neff')

In [ ]:
plt.plot(wg_widths, ngs_te0)
plt.xlabel('wg_width')
plt.ylabel('ng')

In [ ]:
wg_widths = np.arange(200, 2000, 100)*1e-3
n_tm0 = [gm.find_modes(wg_width=wg_width, ncore=3.55, wg_thickness=.22, parity=mp.EVEN_Y) for wg_width in wg_widths]

In [ ]:
neffs_tm0 = [ni['neff'] for ni in n_tm0]
ngs_tm0 = [ni['ng'] for ni in n_tm0]

plt.plot(wg_widths, neffs_te0, '.-', label='TE0')
plt.plot(wg_widths, neffs_tm0, '.-', label='TM0')
plt.xlabel('wg_width')
plt.ylabel('neff')
plt.legend()

# Rib waveguides

In [ ]:
# plot TE mode (plot_mode_order=1)
r = gm.find_modes(wg_width=0.5, ncore=3.55, wg_thickness=.22, slab_thickness=90e-3, res=128)
gm.plot_modes(r['mode_solver'])
r

In [ ]:
neff, ng  = r['neff'], r['ng']
print(neff, ng)

In [ ]:
resolutions = np.linspace(10, 100, 50)
neffs = []
ngs= []

for res in resolutions:
    r = gm.find_modes(wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=res)
    ngs.append(r['ng'])
    neffs.append(r['neff'])

## Convergence tests

Before launching a set of simulations you need to make sure you have the correct simulation settings:

- res: resolution
- sx: Size of the simulation region in the x-direction (default=4.0)
- sy: Size of the simulation region in the y-direction (default=4.0)


In [ ]:
plt.plot(resolutions, ngs, 'o-')
plt.ylabel('ng')
plt.xlabel('resolution (pixels/um)')

In [ ]:
plt.plot(resolutions, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('resolution (pixels/um)')

In [ ]:
sxs = np.linspace(4, 6, 6)
neffs = []
ngs= []

for sx in sxs:
    r = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=20, sx=sx
    )
    ngs.append(r['ng'])
    neffs.append(r['neff'])

In [ ]:
plt.plot(sxs, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in x(um)')

In [ ]:
sys = np.linspace(2, 6, 6)
neffs = []
ngs= []

for sy in sys:
    r = gm.find_modes(
        wg_width=0.5, ncore=3.5, nclad=1.44, wg_thickness=.22, res=20, sy=sy
    )
    ngs.append(r['ng'])
    neffs.append(r['neff'])

In [ ]:
plt.plot(sxs, neffs, 'o-')
plt.ylabel('neff')
plt.xlabel('simulation size in y (um)')

## Dispersion

In [ ]:
from gmeep.find_neff_ng_dw_dh import plot_neff_ng_dw_dh
import pandas as pd
import pathlib
from scipy.interpolate import interp2d
import numpy as np
import gmeep as gm

In [ ]:
plot_neff_ng_dw_dh(with_dispersion=True)

In [ ]:
plot_neff_ng_dw_dh(with_dispersion=False)